In [64]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, KFold, StratifiedKFold

In [2]:
df = pd.read_csv("/content/drive/MyDrive/SML_hw_1/train.csv")

In [3]:
df

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity
0,0007de18844b0dbbb5e1f607da0606e0,0,1,1,1,0,0,1,0,0,0,0,0,63
1,0009c66b9439883ba2750fb825e1d7db,0,1,1,0,0,0,0,0,0,0,0,0,42
2,0013fd999caf9a3efe1352ca1b0d937e,0,1,1,1,0,0,0,0,1,1,0,0,28
3,0018df346ac9c1d8413cfcc888ca8246,0,1,1,1,0,0,0,0,0,0,0,0,15
4,001dc955e10590d3ca4673f034feeef2,0,0,0,1,0,0,1,0,0,0,0,0,72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9880,ffbfa0383c34dc513c95560d6e1fdb57,0,0,0,1,0,0,0,0,0,0,0,1,15
9881,ffcc8532d76436fc79e50eb2e5238e45,0,1,1,1,0,0,0,0,0,0,0,0,70
9882,ffdf2e8673a1da6fb80342fa3b119a20,0,1,1,1,0,0,0,0,1,1,0,0,20
9883,fff19e2ce11718548fa1c5d039a5192a,0,1,1,1,0,0,0,0,1,0,0,0,20


In [4]:
cuts=pd.cut(df["Pawpularity"],[0,25,50,75,100],labels=[1,2,3,4])
df["target"]=cuts

In [5]:
df

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity,target
0,0007de18844b0dbbb5e1f607da0606e0,0,1,1,1,0,0,1,0,0,0,0,0,63,3
1,0009c66b9439883ba2750fb825e1d7db,0,1,1,0,0,0,0,0,0,0,0,0,42,2
2,0013fd999caf9a3efe1352ca1b0d937e,0,1,1,1,0,0,0,0,1,1,0,0,28,2
3,0018df346ac9c1d8413cfcc888ca8246,0,1,1,1,0,0,0,0,0,0,0,0,15,1
4,001dc955e10590d3ca4673f034feeef2,0,0,0,1,0,0,1,0,0,0,0,0,72,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9880,ffbfa0383c34dc513c95560d6e1fdb57,0,0,0,1,0,0,0,0,0,0,0,1,15,1
9881,ffcc8532d76436fc79e50eb2e5238e45,0,1,1,1,0,0,0,0,0,0,0,0,70,3
9882,ffdf2e8673a1da6fb80342fa3b119a20,0,1,1,1,0,0,0,0,1,1,0,0,20,1
9883,fff19e2ce11718548fa1c5d039a5192a,0,1,1,1,0,0,0,0,1,0,0,0,20,1


In [6]:
y = df["target"]

In [7]:
X = df.drop(["Pawpularity", "Id","target"], axis = 1)

In [8]:
y.value_counts()

2    5250
1    2627
3    1329
4     679
Name: target, dtype: int64

## LDA

In [9]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
LDA_transform = LDA(n_components=3)
LDA_transform.fit(X, y)
features_new = LDA_transform.transform(X)

# Print the number of features
print('Original feature #:', X.shape[1])
print('Reduced feature #:', features_new.shape[1])

# Print the ratio of explained variance
print(LDA_transform.explained_variance_ratio_)

Original feature #: 12
Reduced feature #: 3
[0.60490878 0.28577924 0.10931198]


In [10]:
features_new.shape

(9885, 3)

In [11]:
X_LDA = pd.DataFrame(features_new, columns = ['X1','X2','X3'])

## PCA

In [12]:
from sklearn.decomposition import PCA
pca = PCA(n_components=8)
principalComponents = pca.fit_transform(X)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2','principal component 3','principal component 4','principal component 5', 'principal component 6','principal component 7','principal component 8'])

In [13]:
principalDf

,principal component 1,principal component 2,principal component 3,principal component 4,principal component 5,principal component 6,principal component 7,principal component 8
0,0.047108,-0.311538,0.461716,-0.732216,0.204464,0.090231,0.011164,0.109446
1,0.080308,-0.257019,0.539046,0.419956,-0.402315,-0.391238,0.145302,-0.089318
2,-0.847216,0.869587,-0.060273,-0.088875,-0.078299,-0.050394,0.026072,-0.019874
3,-0.141230,-0.347407,-0.129741,-0.022585,-0.054000,-0.044104,-0.065038,-0.023508
4,1.104391,0.324840,0.147222,-0.728335,-0.137728,0.576510,-0.127115,0.123214
...,...,...,...,...,...,...,...,...
9880,1.143077,0.432699,-0.592003,0.038379,0.120487,-0.002341,-0.082307,0.004451
9881,-0.141230,-0.347407,-0.129741,-0.022585,-0.054000,-0.044104,-0.065038,-0.023508
9882,-0.847216,0.869587,-0.060273,-0.088875,-0.078299,-0.050394,0.026072,-0.019874
9883,-0.507765,0.245814,-0.174166,-0.041709,-0.187154,-0.116400,0.086866,0.654165


In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

## Feature Selection

In [15]:
from sklearn.feature_selection import VarianceThreshold
sel_variance_threshold = VarianceThreshold() 
X_train_remove_variance = sel_variance_threshold.fit_transform(X_train)
print(X_train_remove_variance.shape)

(7413, 12)


In [16]:
from sklearn.feature_selection import SelectKBest, chi2, f_classif, mutual_info_classif, SelectFromModel
selector = SelectKBest(f_classif, k =2)
selector.fit(X, y)
X.columns[selector.get_support()]

Index(['Eyes', 'Group'], dtype='object')

In [17]:
selector = SelectKBest(chi2, k =8)
selector.fit(X, y)
X.columns[selector.get_support()]

Index(['Subject Focus', 'Eyes', 'Face', 'Accessory', 'Group', 'Human',
       'Occlusion', 'Blur'],
      dtype='object')

In [18]:
selector = SelectKBest(mutual_info_classif, k =8)
selector.fit(X, y)
X.columns[selector.get_support()]

Index(['Eyes', 'Face', 'Action', 'Accessory', 'Collage', 'Human', 'Info',
       'Blur'],
      dtype='object')

In [19]:
from sklearn.linear_model import LogisticRegression
model_logistic = LogisticRegression(solver='saga', multi_class='multinomial', max_iter=10000, penalty='l1')
sel_model_logistic = SelectFromModel(estimator=model_logistic)
X_train_sfm_l1 = sel_model_logistic.fit_transform(X_train, y_train)
X_train.columns[sel_model_logistic.get_support()]

Index(['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 'Group',
       'Collage', 'Human', 'Occlusion', 'Info', 'Blur'],
      dtype='object')

## From above, we can conclude that top 8 features would be 'Subject Focus', 'Eyes', 'Face', 'Accessory', 'Group', 'Human', 'Occlusion', 'Blur'

## Using SMOTE for data transformation

In [20]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X_train, y_train)

In [21]:
X_res

,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur
0,0,1,1,0,0,0,1,1,0,0,1,0
1,0,1,1,1,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,0,1
3,0,1,1,1,0,0,0,0,0,0,0,0
4,0,1,1,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
15747,0,0,0,1,0,0,0,0,0,0,0,0
15748,0,1,1,1,0,0,0,0,0,0,0,0
15749,0,1,1,0,0,0,0,0,0,0,0,0
15750,0,1,1,1,0,0,0,0,0,0,0,0


In [22]:
y_res.value_counts()

4    3938
3    3938
2    3938
1    3938
Name: target, dtype: int64

## Using random sampler to balance the data

In [23]:
from sklearn.datasets import make_classification
from imblearn.under_sampling import RandomUnderSampler
undersample = RandomUnderSampler(sampling_strategy={1:600,2:600,3:500,4:489})
X_und, y_und = undersample.fit_resample(X_train, y_train)

In [24]:
X_und

,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur
0,0,0,1,0,0,0,0,0,0,0,0,0
1,0,1,1,1,0,0,0,0,0,0,0,0
2,0,1,1,1,0,0,0,0,1,0,0,0
3,0,1,1,1,0,1,0,0,0,1,1,0
4,0,1,1,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2184,0,1,1,1,0,0,0,0,0,0,0,0
2185,0,1,1,1,0,0,1,0,1,1,0,0
2186,0,1,1,1,0,0,0,0,0,0,0,0
2187,0,0,0,1,0,0,1,1,0,0,1,0


In [25]:
y_und.value_counts()

2    600
1    600
3    500
4    489
Name: target, dtype: int64

## Decision Tree model

In [26]:
dtree_model = DecisionTreeClassifier().fit(X_res, y_res)
dtree_predictions = dtree_model.predict(X_test)

In [27]:
dtree_predictions

array([4, 1, 1, ..., 2, 4, 4])

In [28]:
dtree_model.score(X_test, y_test)

0.20469255663430422

In [29]:
cm = confusion_matrix(y_test, dtree_predictions)

In [30]:
cm

array([[ 84, 116, 102, 340],
       [155, 264, 187, 706],
       [ 34,  76,  52, 166],
       [ 17,  40,  27, 106]])

## Random Forest Classifier 

In [70]:
from sklearn.ensemble import RandomForestClassifier
RFC = RandomForestClassifier()


## Search grid for optimal parameters
rf_param_grid = {"max_depth": [None],
              "max_features": [1, 10],
              "min_samples_split": [2, 10],
              "min_samples_leaf": [1, 10],
              "bootstrap": [False],
              "n_estimators" :[100,200],
              "criterion": ["gini"]}
# For hyper parameter tuning

gsRFC = GridSearchCV(RFC,param_grid = rf_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)

gsRFC.fit(X_und,y_und)

RFC_best = gsRFC.best_estimator_

# Best score
gsRFC.best_score_

Fitting 10 folds for each of 16 candidates, totalling 160 fits


0.28049935067655307

In [73]:
RFC_best

RandomForestClassifier(bootstrap=False, max_features=1, min_samples_leaf=10)

In [83]:
## Fitting the best model
RFC = RandomForestClassifier(bootstrap=False, max_features=1, min_samples_leaf=10, n_estimators=200, criterion="gini")
RFC.fit(X_und, y_und)

RandomForestClassifier(bootstrap=False, max_features=1, min_samples_leaf=10,
                       n_estimators=200)

In [84]:
rfc_pred = adaDTC.predict(X_test)
cm = confusion_matrix(y_test, rfc_pred)
cm

array([[140, 375,  52,  75],
       [288, 782, 114, 128],
       [ 65, 190,  28,  45],
       [ 37, 110,  17,  26]])

In [85]:
RFC.score(X_test, y_test)

0.433252427184466

## Extra tree classifier

In [93]:
from sklearn.ensemble import ExtraTreesClassifier
ExtC = ExtraTreesClassifier()


## Search grid for optimal parameters
ex_param_grid = {"max_depth": [None],
              "max_features": [1, 10],
              "min_samples_split": [2, 10],
              "min_samples_leaf": [1, 10],
              "bootstrap": [False],
              "n_estimators" :[100,200],
              "criterion": ["gini"]}


gsExtC = GridSearchCV(ExtC,param_grid = ex_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)

gsExtC.fit(X_und,y_und)

ExtC_best = gsExtC.best_estimator_

# Best score
gsExtC.best_score_

Fitting 10 folds for each of 16 candidates, totalling 160 fits


0.2827908340664404

In [94]:
ExtC_best

ExtraTreesClassifier(max_features=1, min_samples_leaf=10, min_samples_split=10,
                     n_estimators=200)

In [99]:
ExtC = ExtraTreesClassifier(max_features=1, min_samples_leaf=10, min_samples_split=10,
                     n_estimators=200)
ExtC.fit(X_und, y_und)
ext_pred = ExtC.predict(X_test)

In [100]:
cm = confusion_matrix(y_test, ext_pred)
cm

array([[186, 405,  51,   0],
       [339, 859, 114,   0],
       [ 88, 212,  28,   0],
       [ 51, 120,  19,   0]])

In [101]:
ExtC.score(X_test, y_test)

0.43406148867313915

## AdaBoost Classifier

In [74]:
from sklearn.ensemble import AdaBoostClassifier
DTC = DecisionTreeClassifier()

adaDTC = AdaBoostClassifier(DTC, random_state=7)

ada_param_grid = {"base_estimator__criterion" : ["gini", "entropy"],
              "base_estimator__splitter" :   ["best", "random"],
              "algorithm" : ["SAMME","SAMME.R"],
              "n_estimators" :[1,2],
              "learning_rate":  [0.001, 0.01, 0.1, 0.2]}

gsadaDTC = GridSearchCV(adaDTC,param_grid = ada_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)

gsadaDTC.fit(X_und,y_und)

ada_best = gsadaDTC.best_estimator_

Fitting 10 folds for each of 64 candidates, totalling 640 fits


In [75]:
gsadaDTC.best_score_

0.27091030958066276

In [76]:
ada_best

AdaBoostClassifier(algorithm='SAMME', base_estimator=DecisionTreeClassifier(),
                   learning_rate=0.001, n_estimators=1, random_state=7)

In [78]:
## FItting the best model
adaDTC = AdaBoostClassifier(algorithm='SAMME', base_estimator=DecisionTreeClassifier(),
                   learning_rate=0.001, n_estimators=1, random_state=7)
adaDTC.fit(X_und, y_und)

AdaBoostClassifier(algorithm='SAMME', base_estimator=DecisionTreeClassifier(),
                   learning_rate=0.001, n_estimators=1, random_state=7)

In [79]:
ada_pred = adaDTC.predict(X_test)

In [80]:
cm = confusion_matrix(y_test, ada_pred)
cm

array([[140, 375,  52,  75],
       [288, 782, 114, 128],
       [ 65, 190,  28,  45],
       [ 37, 110,  17,  26]])

In [81]:
adaDTC.score(X_test, y_test)

0.3948220064724919

## SVM model using under sampler data set

In [31]:
from sklearn.svm import SVC
svm_model_linear = SVC(kernel = 'linear').fit(X_und, y_und)
svm_predictions = svm_model_linear.predict(X_test)
 
# model accuracy for X_test 
accuracy = svm_model_linear.score(X_test, y_test)
 
# creating a confusion matrix
cm = confusion_matrix(y_test, svm_predictions)

In [32]:
svm_predictions

array([1, 1, 1, ..., 2, 2, 2])

In [33]:
accuracy

0.45105177993527507

In [34]:
cm

array([[164, 451,  27,   0],
       [321, 934,  57,   0],
       [ 81, 230,  17,   0],
       [ 50, 129,  11,   0]])

## SVM model using SMOTE data set

In [35]:
from sklearn.svm import SVC
svm_model_linear = SVC(kernel = 'linear').fit(X_res, y_res)
svm_predictions = svm_model_linear.predict(X_test)
 
# model accuracy for X_test 
accuracy = svm_model_linear.score(X_test, y_test)
 
# creating a confusion matrix
cm = confusion_matrix(y_test, svm_predictions)

In [36]:
accuracy

0.22289644012944984

In [37]:
cm

array([[ 74, 158,  34, 376],
       [121, 339,  76, 776],
       [ 38,  90,  21, 179],
       [ 16,  43,  14, 117]])

## We can determine by compairing the two data sets that Under Sampling performs best in this scenario

## SVM Hyper parameter tuning

In [71]:
SVMC = SVC(probability=True)
svc_param_grid = {'kernel': ['rbf'], 
                  'gamma': [ 0.001, 0.1, 1],
                  'C': [1, 10, 100, 200]}

gsSVMC = GridSearchCV(SVMC,param_grid = svc_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)

gsSVMC.fit(X_und,y_und)

SVMC_best = gsSVMC.best_estimator_

# Best score
gsSVMC.best_score_

Fitting 10 folds for each of 12 candidates, totalling 120 fits


0.2919253487495287

In [72]:
SVMC_best

SVC(C=100, gamma=0.001, probability=True)

In [87]:
SVMC = SVC(C=100, gamma=0.001, probability=True)
SVMC.fit(X_und, y_und)
svm_pred = SVMC.predict(X_test)
cm = confusion_matrix(y_test, svm_pred)

In [88]:
cm

array([[ 126,  494,   22,    0],
       [ 241, 1022,   49,    0],
       [  61,  252,   15,    0],
       [  40,  141,    9,    0]])

In [90]:
accuracy = SVMC.score(X_test, y_test)

In [91]:
accuracy

0.4704692556634304

## KNN model

In [38]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 7).fit(X_res, y_res)
 
# accuracy on X_test
accuracy = knn.score(X_test, y_test)
 
# creating a confusion matrix
knn_predictions = knn.predict(X_test)
cm = confusion_matrix(y_test, knn_predictions)

In [39]:
accuracy

0.47815533980582525

In [40]:
knn_predictions

array([2, 2, 1, ..., 2, 2, 2])

In [41]:
cm

array([[ 108,  499,   34,    1],
       [ 183, 1061,   61,    7],
       [  54,  261,   12,    1],
       [  24,  152,   13,    1]])

In [103]:
## Hyper parameter tuning
knn_param_grid = {'leaf_size': [1,5,10,20], 
                  'n_neighbors': [1,5,10,20],
                  'p': [1, 2]}

knn = KNeighborsClassifier()
gsKNN = GridSearchCV(knn,param_grid = knn_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)

gsKNN.fit(X_und,y_und)

KNN_best = gsKNN.best_estimator_

# Best score
gsKNN.best_score_

Fitting 10 folds for each of 32 candidates, totalling 320 fits


0.2823090779607054

In [104]:
KNN_best

KNeighborsClassifier(leaf_size=20, n_neighbors=10, p=1)

In [105]:
knn = KNeighborsClassifier(leaf_size=20, n_neighbors=10, p=1).fit(X_res, y_res)
 
# accuracy on X_test
accuracy = knn.score(X_test, y_test)
 
# creating a confusion matrix
knn_predictions = knn.predict(X_test)
cm = confusion_matrix(y_test, knn_predictions)


In [106]:
cm

array([[  78,  562,    0,    2],
       [ 118, 1185,    5,    4],
       [  31,  294,    1,    2],
       [  15,  174,    0,    1]])

In [107]:
accuracy

0.5117313915857605

## After hyper parameter tuning the model has shown increase in accuracy, KNN seems to working better than other models.

## Gaussian model

In [42]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB().fit(X_res, y_res)
gnb_predictions = gnb.predict(X_test)
 
# accuracy on X_test
accuracy = gnb.score(X_test, y_test)

 
# creating a confusion matrix
cm = confusion_matrix(y_test, gnb_predictions)

In [43]:
accuracy

0.13794498381877024

In [44]:
cm

array([[ 61,  59,  94, 428],
       [107, 102, 199, 904],
       [ 36,  28,  48, 216],
       [ 12,  17,  31, 130]])

## XGBOOST CLASSIFIER

In [45]:
xg = XGBClassifier(learning_rate= 0.2113303692287583,
  reg_lambda= 0.0005172374569093787,
  reg_alpha= 0.001273145009879541,
  subsample= 0.12703520389320402,
  colsample_bytree= 0.2566392406542385,
  max_depth= 2,
  random_state=42,
  n_estimators=1000)

In [46]:
xg.fit(X_und,y_und)

XGBClassifier(colsample_bytree=0.2566392406542385,
              learning_rate=0.2113303692287583, max_depth=2, n_estimators=1000,
              objective='multi:softprob', random_state=42,
              reg_alpha=0.001273145009879541, reg_lambda=0.0005172374569093787,
              subsample=0.12703520389320402)

In [47]:
y_pred_xg = xg.predict(X_test)

In [48]:
cm = confusion_matrix(y_test, y_pred_xg)

In [49]:
cm

array([[165, 373,  93,  11],
       [313, 785, 192,  22],
       [ 76, 192,  52,   8],
       [ 42, 106,  39,   3]])

In [50]:
accuracy = xg.score(X_test, y_test)

In [51]:
accuracy

0.4065533980582524

## Gradient Boosting 


In [65]:
from sklearn.ensemble import GradientBoostingClassifier
kfold = StratifiedKFold(n_splits=10)
GBC = GradientBoostingClassifier()
gb_param_grid = {'loss' : ["deviance"],
              'n_estimators' : [100,200],
              'learning_rate': [0.1, 0.01],
              'max_depth': [4, 8],
              'min_samples_leaf': [100,150],
              'max_features': [0.3, 0.1] 
              }
## Hyper parameter tuning
gsGBC = GridSearchCV(GBC,param_grid = gb_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)

gsGBC.fit(X_und,y_und)

GBC_best = gsGBC.best_estimator_

# Best score
gsGBC.best_score_

Fitting 10 folds for each of 72 candidates, totalling 720 fits


KeyboardInterrupt: ignored

## Ridge and Lasso models

In [52]:
from sklearn.linear_model import Ridge, Lasso

In [53]:
ridgeModel = Ridge(alpha=2)
ridgeModel.fit(X_und, y_und)
 
# Evaluating the Ridge Regression model
print(ridgeModel.score(X_test, y_test))

-0.2063690961676512


In [54]:
y_pred_r = ridgeModel.predict(X_test)

In [55]:
#cm = confusion_matrix(y_test, y_pred_r)

In [56]:
lassoModel = Lasso()
lassoModel.fit(X_und, y_und)
 
# Evaluating the Lasso Regression model
print(lassoModel.score(X_test, y_test))

-0.2004982149533412
